In [ ]:
import redis
import numpy as np
r = redis.Redis(host='localhost', port=6379, db=0)
dev = 'computer:ff:ff:ff:ff:ff:ff'

In [ ]:
cpu = np.array(r.json().get(dev, 'cpu_usage')).transpose(1, 0)
mem = np.array(r.json().get(dev, 'mem_usage')).transpose(1, 0)
disk_read = np.array(r.json().get(dev, 'disk_read')).transpose(1, 0)
disk_write = np.array(r.json().get(dev, 'disk_write')).transpose(1, 0)
net = np.array(r.json().get(dev, 'net_capture')).transpose(1, 0)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
output_notebook(INLINE)

In [ ]:
p = figure(title="Simple line example", x_axis_label='Time (s)', y_axis_label='y')
p.line(*mem, color='red' , legend_label="cpu" , line_width=2)
show(p)

In [ ]:
for i in range(1,4):
    p = figure(title=f"Simple line example {i}", x_axis_label='Time (s)', y_axis_label='y')
    nfr, nfw = max(disk_read[i])-min(disk_read[i]), max(disk_write[i])-min(disk_write[i])
    p.line((disk_read[0] -disk_read[0,0])/1e3, (disk_read[i]  -min(disk_read[i] ))/nfr, color='red' , legend_label=f"disk_read[{i}]" , line_width=2)
    p.line((disk_write[0]-disk_read[0,0])/1e3, (disk_write[i] -min(disk_write[i]))/nfw, color='blue', legend_label=f"disk_write[{i}]", line_width=2)
    show(p)
